In [1]:
!pip install pymongo
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28011"
user="GROUP_11"
password="MTM0MzQ5MzI3NjY4ODM1MzQ2ODc2OTMzMjgyNjQ3NjYyNDcyNjUw"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
eu = db.eu

In [4]:
bot_year = 2008
top_year = 2020
country_list = ['NO', 'HR', 'HU', 'CH', 'CZ', 'RO', 'LV', 'GR', 'UK', 'SI', 'LT', 'ES', 'FR', 'IE', 'SE', 'NL',
                'PT', 'PL', 'DK', 'MK', 'DE', 'IT', 'BG', 'CY', 'AT', 'LU', 'BE', 'FI', 'EE', 'SK', 'MT', 'LI',
                'IS']

In [5]:
#eu.create_index([('YEAR',1),('ISO_COUNTRY_CODE',1)],name='year_country_index')

In [6]:
project = {
    '$project':{
        '_id':False,
        'YEAR':True,
        'ISO_COUNTRY_CODE':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'AWARD_VALUE_EURO':True
    }
}

group = {
    '$group':{
        '_id':{'YEAR':'$YEAR','COUNTRY':'$ISO_COUNTRY_CODE','CPV_DIVISION':'$CPV_DIVISION'},
        'AVERAGE':{'$avg':'$AWARD_VALUE_EURO'},
        'MAX':{'$max':'$AWARD_VALUE_EURO'},
        'COUNT':{'$sum':1}
    }
}

sort = {
    '$sort':{
        '_id.YEAR':-1,
        '_id.COUNTRY':1,
        '_id.CPV_DIVISION':1
    }
}

pipeline = [project,group,sort]

year_country_cpv = list(eu.aggregate(pipeline))

year_country_cpv

[{'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '14'},
  'AVERAGE': 4326930.25,
  'MAX': 14700000,
  'COUNT': 9},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '15'},
  'AVERAGE': 1442862.0196296296,
  'MAX': 21150000,
  'COUNT': 137},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '16'},
  'AVERAGE': 578075.0,
  'MAX': 893000,
  'COUNT': 2},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '18'},
  'AVERAGE': 1055716.9766666666,
  'MAX': 6496533,
  'COUNT': 37},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '19'},
  'AVERAGE': 239749.47,
  'MAX': 1287240,
  'COUNT': 17},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '22'},
  'AVERAGE': 1642416.7289999998,
  'MAX': 7500000,
  'COUNT': 118},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '24'},
  'AVERAGE': 809658.6950000001,
  'MAX': 1965000,
  'COUNT': 16},
 {'_id': {'YEAR': 2018, 'COUNTRY': 'AT', 'CPV_DIVISION': '30'},
  'AVERAGE': 3902266.3261290323,
  'MAX': 3949

In [7]:
#eu.drop_index('year_country_index')

In [8]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'NUMBER_OFFERS':True,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

group_cpv_euro_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CPV_VALUE':{'$avg':'$AVERAGE_VALUE'}
    }
}

limit = {
    '$limit':5
}

pipeline_cpv_euro_avg = [match, project, limit, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg = list(eu.aggregate(pipeline_cpv_euro_avg))

agg_cpv_euro_avg[0]['AVERAGE_CPV_VALUE']

858251.6666666666

In [9]:
group_cpv_count = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'COUNT':{'$sum':1}
    }
}

group_cpv_count_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNT':{'$avg':'$COUNT'}
    }
}

pipeline_cpv_count = [match, project, group_cpv_count, group_cpv_count_2]

agg_cpv_count = list(eu.aggregate(pipeline_cpv_count))

agg_cpv_count[0]['AVERAGE_COUNT']

120942.86956521739

In [10]:
group_cpv_offer_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_OFFERS':{'$avg':'$NUMBER_OFFERS'}
    }
}

group_cpv_offer_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_CPV_OFFERS':{'$avg':'$AVERAGE_OFFERS'}
    }
}

pipeline_cpv_offer_avg = [match, project, group_cpv_offer_avg, group_cpv_offer_avg_2]

agg_cpv_offer_avg = list(eu.aggregate(pipeline_cpv_offer_avg))

agg_cpv_offer_avg[0]['AVERAGE_CPV_OFFERS']

7.513253256774548

In [11]:
match_cpv_euro_avg_y_eu = {
    '$match':{
        'B_EU_FUNDS':'Y'
    }
}

pipeline_cpv_euro_avg_y_eu = [match, match_cpv_euro_avg_y_eu, project, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg_y_eu = list(eu.aggregate(pipeline_cpv_euro_avg_y_eu))

agg_cpv_euro_avg_y_eu[0]['AVERAGE_CPV_VALUE']

1.1953484030619935e+22

In [12]:
match_cpv_euro_avg_n_eu = {
    '$match':{
        'B_EU_FUNDS':'N'
    }
}

pipeline_cpv_euro_avg_n_eu = [match, match_cpv_euro_avg_n_eu, project, group_cpv_euro_avg, group_cpv_euro_avg_2]

agg_cpv_euro_avg_n_eu = list(eu.aggregate(pipeline_cpv_euro_avg_n_eu))

agg_cpv_euro_avg_n_eu[0]['AVERAGE_CPV_VALUE']

1.3825420111135592e+19

In [13]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]}
    }
}

group_cpv_count = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'COUNT':{'$sum':1}
    }
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'count':'$COUNT'
    }
}

project_3 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'count':True
    }
}

pipeline = [match, project, group_cpv_count, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'count': 252972, 'cpv': 'Food, beverages, tobacco and related products'},
 {'count': 204709,
  'cpv': 'Business services: law, marketing, consulting, recruitment, printing and security'},
 {'count': 26641,
  'cpv': 'Security, fire-fighting, police and defence equipment'},
 {'count': 3155, 'cpv': 'Services related to the oil and gas industry'},
 {'count': 14389,
  'cpv': 'Administration, defence and social security services'},
 {'count': 44758,
  'cpv': 'Radio, television, communication, telecommunication and related equipment'},
 {'count': 11924, 'cpv': 'Real estate services'},
 {'count': 13465, 'cpv': 'Public utilities'},
 {'count': 131271, 'cpv': 'Education and training services'},
 {'count': 243228,
  'cpv': 'Sewage, refuse, cleaning and environmental services'},
 {'count': 99613,
  'cpv': 'IT services: consulting, software development, Internet and support'},
 {'count': 1777086,
  'cpv': 'Medical equipments, pharmaceuticals and personal care products'},
 {'count': 183605, 'cpv': 

In [14]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'VALUE_EURO':True
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':-1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_3 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, project, group_cpv_euro_avg, sort, limit, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 1.1102520637497268e+22,
  'cpv': 'Sewage, refuse, cleaning and environmental services'},
 {'avg': 4.283694829580493e+20,
  'cpv': 'Office and computing machinery, equipment and supplies except furniture and software packages'},
 {'avg': 1.0979761462251373e+19,
  'cpv': 'Administration, defence and social security services'},
 {'avg': 3.572801959007498e+17,
  'cpv': 'Business services: law, marketing, consulting, recruitment, printing and security'},
 {'avg': 233961466170367.25, 'cpv': 'Education and training services'}]

In [15]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'VALUE_EURO':True
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_3 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, project, group_cpv_euro_avg, sort, limit, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 1666574.1648330106, 'cpv': 'Agricultural machinery'},
 {'avg': 1672027.3621667938,
  'cpv': 'Musical instruments, sport goods, games, toys, handicraft, art materials and accessories'},
 {'avg': 1779413.0967100735,
  'cpv': 'Recreational, cultural and sporting services'},
 {'avg': 1971553.785299904,
  'cpv': 'Leather and textile fabrics, plastic and rubber materials'},
 {'avg': 2166225.6565261814, 'cpv': 'Collected and purified water'}]

In [16]:
match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list},
            'B_EU_FUNDS':'Y'
        }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'VALUE_EURO':True
    }
}

group_cpv_euro_avg = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'AVERAGE_VALUE':-1
    }
}

limit = {
    '$limit':5
}

lookup = {
    '$lookup':{
        'from':'cpv',
        'localField':'_id',
        'foreignField':'cpv_division',
        'as':'CPV'
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'cpv':{ "$arrayElemAt": [ "$CPV", 0] },
        'avg':'$AVERAGE_VALUE'
    }
}

project_3 = {
    '$project':{
        'cpv':'$cpv.cpv_division_description',
        'avg':True
    }
}

pipeline = [match, project, group_cpv_euro_avg, sort, limit, lookup, project_2, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'avg': 5.498602649357064e+23,
  'cpv': 'Sewage, refuse, cleaning and environmental services'},
 {'avg': 381753911395855.75, 'cpv': 'Education and training services'},
 {'avg': 91000076123835.94,
  'cpv': 'Research and development services and related consultancy services'},
 {'avg': 56250333795.48817, 'cpv': 'Construction work'},
 {'avg': 100058341.25789362,
  'cpv': 'Administration, defence and social security services'}]

In [17]:
match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list}
        }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'ISO_COUNTRY_CODE':True,
        'VALUE_EURO':True
    }
}

group_cpv_country_euro_avg = {
    '$group':{
        '_id':{'CPV_DIVISION':'$CPV_DIVISION','COUNTRY':'$ISO_COUNTRY_CODE'},
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        'COUNTRY':1,
        'AVERAGE_VALUE':-1
    }
}

pipeline = [match, project, group_cpv_euro_avg, limit]#, sort]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'_id': '44', 'AVERAGE_VALUE': 11907487.169259189},
 {'_id': '41', 'AVERAGE_VALUE': 2166225.6565261814},
 {'_id': '51', 'AVERAGE_VALUE': 6313617.474037876},
 {'_id': '30', 'AVERAGE_VALUE': 4.283694829580493e+20},
 {'_id': '73', 'AVERAGE_VALUE': 34314116157954.98}]

In [18]:
cpv = '50'
limit = {
    '$limit':5
}

In [19]:
project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'YEAR':True,
        'ISO_COUNTRY_CODE':True,
        'VALUE_EURO':True
    }
}

match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list},
            'CPV_DIVISION': {'$eq': cpv}
        }
}

bucket = {
    '$bucket':{
        'groupBy':'$VALUE_EURO',
        'boundaries':[0,100000,200000,300000,400000,500000,600000,700000,800000,900000,1000000],
        'default':'>1000000',
        'output':{
            'count':{'$sum':1}
        }
    }
}

project_2 = {
    '$project':{
        '_id':False,
        'bucket':'$_id',
        'count':True
    }
}

pipeline = [project, match, bucket, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'count': 16658, 'bucket': 0},
 {'count': 11690, 'bucket': 100000},
 {'count': 9105, 'bucket': 200000},
 {'count': 6471, 'bucket': 300000},
 {'count': 4597, 'bucket': 400000},
 {'count': 3989, 'bucket': 500000},
 {'count': 3172, 'bucket': 600000},
 {'count': 2128, 'bucket': 700000},
 {'count': 1992, 'bucket': 800000},
 {'count': 1597, 'bucket': 900000},
 {'count': 122206, 'bucket': '>1000000'}]

In [26]:
match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list}
        }
}

project = {
    '$project':{
        '_id':False,
        'CPV_DIVISION':{'$substr':['$CPV',0,2]},
        'AWARD_VALUE_EURO':True,
        'VALUE_EURO':True,
        'DT_DISPATCH_DATE':{'$dateFromString':{'dateString':'$DT_DISPATCH'}},
        'DT_AWARD_DATE':{'$dateFromString':{'dateString':'$DT_AWARD'}},
    }
}

project_2 = {
    '$project':{
        'CPV_DIVISION':True,
        'TIME_DIFFERENCE':{'$subtract':['$DT_DISPATCH_DATE','$DT_AWARD_DATE']},
        'VALUE_DIFFERENCE':{'$subtract':['$AWARD_VALUE_EURO','$VALUE_EURO']}
    }
}

group_cpv_difference = {
    '$group':{
        '_id':'$CPV_DIVISION',
        'AVG_TIME_DIFFERENCE':{'$avg':'$TIME_DIFFERENCE'},
        'AVG_VALUE_DIFFERENCE':{'$avg':'$VALUE_DIFFERENCE'}
    }
}

project_3 = {
    '$project':{
        '_id':False,
        'cpv':'$_id',
        'time_difference':'$AVG_TIME_DIFFERENCE',
        'value_difference':'$AVG_VALUE_DIFFERENCE'
    }
}

pipeline = [match, project, project_2, group_cpv_difference, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'cpv': '63',
  'time_difference': 3738115641.978839,
  'value_difference': -2286338.36504143},
 {'cpv': '45',
  'time_difference': 4594982684.710538,
  'value_difference': -32733941041.76896},
 {'cpv': '38',
  'time_difference': 4648555504.456788,
  'value_difference': -1624922.4882059107},
 {'cpv': '60',
  'time_difference': 4791031309.565406,
  'value_difference': -219833343.56677574},
 {'cpv': '98',
  'time_difference': 4727401998.667555,
  'value_difference': -3584229.4669107217},
 {'cpv': '39',
  'time_difference': 3957434989.1007886,
  'value_difference': -5480490.594692337},
 {'cpv': '77',
  'time_difference': 3284324326.286181,
  'value_difference': -1783870.9933145395},
 {'cpv': '93',
  'time_difference': 3527022537.0990906,
  'value_difference': -6074803.222549212},
 {'cpv': '48',
  'time_difference': 3593156295.933366,
  'value_difference': -585311567642.2299},
 {'cpv': '16',
  'time_difference': 5004415047.918753,
  'value_difference': -819891.8037723078},
 {'cpv': '92',


In [28]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

group_country_euro_avg = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE_VALUE':{'$avg':'$VALUE_EURO'}
    }
}

group_country_euro_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNTRY_VALUE':{'$avg':'$AVERAGE_VALUE'}
    }
}

pipeline_country_euro_avg = [match, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg = list(eu.aggregate(pipeline_country_euro_avg))

agg_country_euro_avg[0]['AVERAGE_COUNTRY_VALUE']

7.156703471406817e+20

In [29]:
group_country_count = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'COUNT':{'$sum':1}
    }
}

group_country_count_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNT':{'$avg':'$COUNT'}
    }
}

pipeline_country_count = [match, group_country_count, group_country_count_2]

agg_country_count = list(eu.aggregate(pipeline_country_count))

agg_country_count[0]['AVERAGE_COUNT']

168587.0303030303

In [30]:
group_country_offer_avg = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE_OFFERS':{'$avg':'$NUMBER_OFFERS'}
    }
}

group_country_offer_avg_2 = {
    '$group':{
        '_id':None,
        'AVERAGE_COUNTRY_OFFERS':{'$avg':'$AVERAGE_OFFERS'}
    }
}

pipeline_country_offer_avg = [match, group_country_offer_avg, group_country_offer_avg_2]

agg_country_offer_avg = list(eu.aggregate(pipeline_country_offer_avg))

agg_country_offer_avg[0]['AVERAGE_COUNTRY_OFFERS']

12.580563334769861

In [31]:
match_country_euro_avg_y_eu = {
    '$match':{
        'B_EU_FUNDS':'Y'
    }
}

pipeline_country_euro_avg_y_eu = [match, match_country_euro_avg_y_eu, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg_y_eu = list(eu.aggregate(pipeline_country_euro_avg_y_eu))

agg_country_euro_avg_y_eu[0]['AVERAGE_COUNTRY_VALUE']

3.952232841686021e+21

In [32]:
match_country_euro_avg_n_eu = {
    '$match':{
        'B_EU_FUNDS':'N'
    }
}

pipeline_country_euro_avg_n_eu = [match, match_country_euro_avg_n_eu, group_country_euro_avg, group_country_euro_avg_2]

agg_country_euro_avg_n_eu = list(eu.aggregate(pipeline_country_euro_avg_n_eu))

agg_country_euro_avg_n_eu[0]['AVERAGE_COUNTRY_VALUE']

5.439355769457974e+18